In [6]:
from gensim import corpora, models, similarities
from sklearn.datasets import fetch_20newsgroups as getData
from sklearn.model_selection import train_test_split
from collections import defaultdict
import re
import numpy as np

In [7]:
corpus = getData(subset = 'train', remove = ('headers', 'footers', 'quotes'))
X = corpus.data
y = corpus.target
y_names = corpus.target_names

In [12]:
print(X[0])
print(len(X))
print(y_names[y[0]])

I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.
11314
rec.autos


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9, shuffle = True)

In [9]:
stoplist = set('for a of the and in'.split(' '))
texts = [[word for word in re.split('\W+', doc.lower()) if word not in stoplist] 
        for doc in X_train]

In [13]:
print(len(texts))

10182


In [14]:
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token]+=1

cutOffValue = 10
processed_corpus = [[token for token in text if frequency[token]> cutOffValue]
                   for text in texts]

In [16]:
dictionary = corpora.Dictionary(processed_corpus)
print(dictionary)

Dictionary<12407 unique tokens: ['', '0', '1', '1x', '2']...>


In [19]:
bow_corpus = [dictionary.doc2bow(text) for text in processed_corpus]

In [20]:
print(bow_corpus[100])

[(0, 1), (10, 1), (26, 1), (47, 1), (48, 1), (66, 1), (75, 3), (87, 1), (107, 1), (130, 1), (133, 1), (143, 1), (146, 1), (166, 1), (194, 1), (222, 1), (230, 1), (369, 1), (392, 1), (502, 1), (533, 1), (613, 1), (713, 2), (729, 1), (731, 1), (799, 1), (1202, 1), (1345, 1), (1700, 1), (2383, 1), (2972, 1), (3019, 1), (3044, 1), (3045, 1)]


In [21]:
model = models.TfidfModel(bow_corpus)

In [23]:
index = similarities.SparseMatrixSimilarity(model[bow_corpus], num_features=len(dictionary))

In [43]:
t = np.random.randint(len(X_test))


query_document = re.split('\W+', X_test[t].lower())


query_bow = dictionary.doc2bow(query_document)

sims = index[model[query_bow]]

docNumber = sorted(enumerate(sims), key = lambda x: x[1], reverse = True)[0][0]

print('Predicted:', y_names[y_train[docNumber]])
print('Ground Truth:', y_names[y_test[t]])

Predicted: comp.sys.ibm.pc.hardware
Ground Truth: comp.sys.ibm.pc.hardware
